##Data pre-processing


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#loading data with financial ratios data
import pandas as pd
dtype = {'gvkey': 'int32', 'evm': 'float32', 'GProf': 'float32', 'cfm': 'float32', 'quick_ratio': 'float32',
         'roa': 'float32','roe': 'float32','roce': 'float32','equity_invcap': 'float32','debt_invcap': 'float32','debt_at': 'float32',
         'CAPEI': 'float32', 'bm': 'float32','ps': 'float32','pcf': 'float32','totdebt_invcap': 'float32','rd_sale': 'float32', 'sale_invcap': 'float32',
         'debt_ebitda': 'float32', 'debt_assets': 'float32', 'de_ratio': 'float32', 'ptb': 'float32',
         'divyield': 'str', 'adate': 'str', 'qdate': 'str', 'public_date': 'str'}
df = pd.read_csv('/content/drive/My Drive/Project_Bank/Data/factors.csv.gz', compression='gzip',
                   error_bad_lines=False, parse_dates = ['adate', 'qdate', 'public_date'], dtype = dtype)
df.head()

,gvkey,adate,qdate,public_date,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield
0,1004,1978-05-31,1978-11-30,1979-01-31,10.409,1.236,5.802,0.179,41.009998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.983,3.17%
1,1004,1978-05-31,1978-11-30,1979-02-28,10.192,1.236,5.802,0.176,40.155998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.963,3.23%
2,1004,1978-05-31,1978-11-30,1979-03-31,10.517,1.236,5.802,0.181,41.437000,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.993,3.13%
3,1004,1978-05-31,1979-02-28,1979-04-30,9.320,1.066,5.657,0.157,9.357000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.878,3.45%
4,1004,1978-05-31,1979-02-28,1979-05-31,8.811,1.066,5.657,0.149,8.846000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.830,4.24%


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893868 entries, 0 to 893867
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   gvkey           893868 non-null  int32         
 1   adate           885813 non-null  datetime64[ns]
 2   qdate           888303 non-null  datetime64[ns]
 3   public_date     893868 non-null  datetime64[ns]
 4   CAPEI           841021 non-null  float32       
 5   bm              862661 non-null  float32       
 6   evm             880898 non-null  float32       
 7   ps              869393 non-null  float32       
 8   pcf             879340 non-null  float32       
 9   cfm             858368 non-null  float32       
 10  roa             879272 non-null  float32       
 11  roe             851843 non-null  float32       
 12  roce            873458 non-null  float32       
 13  GProf           884357 non-null  float32       
 14  equity_invcap   881771 non-null  flo

In [ ]:
#plt.figure(figsize = (10,10))
#sns.heatmap(df.isna(), cbar = 'False')

In [ ]:
df['gvkey'].nunique()

4482

In [ ]:
#loading price data
import pandas as pd
dtype_p = {'gvkey': 'int32', 'iid': 'str', 'datadate': 'str', 'tic': 'str', 'ajexm': 'float32', 'prccm': 'float32', 'cshom': 'float32', 'trt1m': 'float32', 'ggroup': 'str', 'gind': 'str', 'gsector': 'str', 'ggsubind': 'int32' }
df_price = pd.read_csv('/content/drive/My Drive/Project_Bank/Data/price_data.csv.gz', compression='gzip',
                   error_bad_lines=False, parse_dates = ['datadate'], dtype = dtype_p)
df_price = df_price.rename(columns = {'datadate':'public_date'})
df_price['prccm'].replace(0,0.0001, inplace = True) # replaced 0 with very small value to prevent division by zero when calculating returns
df_price['ajexm'].replace(0,1, inplace = True) # replaced 0 with 1 i.e there is no change in the price, if we kept zero then div by zero error will pop up
df_price.head()

,gvkey,iid,public_date,tic,ajexm,prccm,trt1m,cshom,ggroup,gind,gsector,gsubind
0,1004,01,1979-01-31,AIR,4.2188,12.000,15.6627,NaN,2010,201010,20,20101010.0
1,1004,01,1979-02-28,AIR,4.2188,11.750,-1.2917,NaN,2010,201010,20,20101010.0
2,1004,01,1979-03-31,AIR,4.2188,12.125,3.1915,NaN,2010,201010,20,20101010.0
3,1004,01,1979-04-30,AIR,4.2188,11.000,-9.2783,NaN,2010,201010,20,20101010.0
4,1004,01,1979-05-31,AIR,4.2188,10.375,-4.6818,NaN,2010,201010,20,20101010.0


In [ ]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989408 entries, 0 to 1989407
Data columns (total 12 columns):
 #   Column       Dtype         
---  ------       -----         
 0   gvkey        int32         
 1   iid          object        
 2   public_date  datetime64[ns]
 3   tic          object        
 4   ajexm        float32       
 5   prccm        float32       
 6   trt1m        float32       
 7   cshom        float32       
 8   ggroup       object        
 9   gind         object        
 10  gsector      object        
 11  gsubind      float64       
dtypes: datetime64[ns](1), float32(4), float64(1), int32(1), object(5)
memory usage: 144.2+ MB


In [ ]:
df_price['ggroup'] = df_price['ggroup'].apply(lambda x: str(x)[2:] if x != 'NaN' else np.NaN) 
df_price['gind'] = df_price['gind'].apply(lambda x: str(x)[4:] if x != 'NaN' else np.NaN)
df_price['gsubind'] = df_price['gsubind'].apply(lambda x: str(x).split('.')[0][6:] if x != 'NaN' else np.NaN)  
df_price.head()

,gvkey,iid,public_date,tic,ajexm,prccm,trt1m,cshom,ggroup,gind,gsector,gsubind
0,1004,01,1979-01-31,AIR,4.2188,12.000,15.6627,NaN,10,10,20,10
1,1004,01,1979-02-28,AIR,4.2188,11.750,-1.2917,NaN,10,10,20,10
2,1004,01,1979-03-31,AIR,4.2188,12.125,3.1915,NaN,10,10,20,10
3,1004,01,1979-04-30,AIR,4.2188,11.000,-9.2783,NaN,10,10,20,10
4,1004,01,1979-05-31,AIR,4.2188,10.375,-4.6818,NaN,10,10,20,10


In [ ]:
df_price['adj_prccm'] = df_price['prccm'].div(df_price['ajexm']) #adjusting price
df_price['adj_cshom'] = df_price['cshom'].mul(df_price['ajexm']) #adjusting shares outstanding
df_price.head()

,gvkey,iid,public_date,tic,ajexm,prccm,trt1m,cshom,ggroup,gind,gsector,gsubind,adj_prccm,adj_cshom
0,1004,01,1979-01-31,AIR,4.2188,12.000,15.6627,NaN,10,10,20,10,2.844411,NaN
1,1004,01,1979-02-28,AIR,4.2188,11.750,-1.2917,NaN,10,10,20,10,2.785152,NaN
2,1004,01,1979-03-31,AIR,4.2188,12.125,3.1915,NaN,10,10,20,10,2.874040,NaN
3,1004,01,1979-04-30,AIR,4.2188,11.000,-9.2783,NaN,10,10,20,10,2.607377,NaN
4,1004,01,1979-05-31,AIR,4.2188,10.375,-4.6818,NaN,10,10,20,10,2.459230,NaN


In [ ]:
keys = list(df_price['gvkey'].unique())

In [ ]:
#takes a lot of time
#generating 1 month future returns
df2 = pd.DataFrame()
for key in keys:
  k = df_price[df_price['gvkey'] == int(key)]
  k['fut_1m_r'] = k['adj_prccm'].pct_change().shift(-1)
  k['1yr_ret_vol'] = k['trt1m'].div(100).rolling(12).std()
  k['trt1m'] = k['trt1m'].div(100)
  df2 = pd.concat([df2,k])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
df2.shape

(1119276, 16)

In [ ]:
df2 = df2.set_index(['public_date','gvkey']).join(df.set_index(['public_date','gvkey']), how = 'left') #merging ratios and price data on (date, gvkey)

In [ ]:
df.head()

,gvkey,adate,qdate,public_date,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield
0,1004,1978-05-31,1978-11-30,1979-01-31,10.409,1.236,5.802,0.179,41.009998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.983,3.17%
1,1004,1978-05-31,1978-11-30,1979-02-28,10.192,1.236,5.802,0.176,40.155998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.963,3.23%
2,1004,1978-05-31,1978-11-30,1979-03-31,10.517,1.236,5.802,0.181,41.437000,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.993,3.13%
3,1004,1978-05-31,1979-02-28,1979-04-30,9.320,1.066,5.657,0.157,9.357000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.878,3.45%
4,1004,1978-05-31,1979-02-28,1979-05-31,8.811,1.066,5.657,0.149,8.846000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.830,4.24%


In [ ]:
df = df.reset_index()

In [ ]:
df.to_pickle('raw_data.gz', compression = 'gzip')

##DATA FILLING

In [ ]:
df = pd.read_pickle('/content/drive/My Drive/Project_Bank/Data/raw_data.gz', compression = 'gzip')
df.head()

,index,gvkey,adate,qdate,public_date,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield
0,0,1004,1978-05-31,1978-11-30,1979-01-31,10.409,1.236,5.802,0.179,41.009998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.983,3.17%
1,1,1004,1978-05-31,1978-11-30,1979-02-28,10.192,1.236,5.802,0.176,40.155998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.963,3.23%
2,2,1004,1978-05-31,1978-11-30,1979-03-31,10.517,1.236,5.802,0.181,41.437000,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.993,3.13%
3,3,1004,1978-05-31,1979-02-28,1979-04-30,9.320,1.066,5.657,0.157,9.357000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.878,3.45%
4,4,1004,1978-05-31,1979-02-28,1979-05-31,8.811,1.066,5.657,0.149,8.846000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.830,4.24%


In [ ]:
#dividend yield was a string with % sign, converted to float 
df['divyield'] = df['divyield'].apply(lambda x: str(x)[:-1] if x != 'NaN' else np.NaN)
df['divyield'] = df['divyield'].apply(lambda x: float(x) if x != 'na' else np.NaN)
df.head() 

,index,gvkey,adate,qdate,public_date,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield
0,0,1004,1978-05-31,1978-11-30,1979-01-31,10.409,1.236,5.802,0.179,41.009998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.983,3.17
1,1,1004,1978-05-31,1978-11-30,1979-02-28,10.192,1.236,5.802,0.176,40.155998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.963,3.23
2,2,1004,1978-05-31,1978-11-30,1979-03-31,10.517,1.236,5.802,0.181,41.437000,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.993,3.13
3,3,1004,1978-05-31,1979-02-28,1979-04-30,9.320,1.066,5.657,0.157,9.357000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.878,3.45
4,4,1004,1978-05-31,1979-02-28,1979-05-31,8.811,1.066,5.657,0.149,8.846000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.830,4.24


In [ ]:
dates = list(df['public_date'].unique())
gvkeys = list(df['gvkey'].unique())

In [ ]:
#import seaborn as sns

#plt.figure(figsize = (15,15))
#sns.heatmap(df_fill.isna(), cbar = False)

In [ ]:
"""
This takes a very long time,
forward fill first for time series per gvkey,
backfill(this will fill only where first value 
was nan as these won't be filled by forward fill) 
to fill rest nans.
"""
df_fill = pd.DataFrame()

for key in gvkeys:
  slc = df[df['gvkey'] == key]
  slc = slc.fillna(method = 'ffill')
  slc = slc.fillna(method = 'bfill')
  df_fill = pd.concat([df_fill, slc])

df_fill.head()

,index,gvkey,adate,qdate,public_date,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield
0,0,1004,1978-05-31,1978-11-30,1979-01-31,10.409,1.236,5.802,0.179,41.009998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.983,3.17
1,1,1004,1978-05-31,1978-11-30,1979-02-28,10.192,1.236,5.802,0.176,40.155998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.963,3.23
2,2,1004,1978-05-31,1978-11-30,1979-03-31,10.517,1.236,5.802,0.181,41.437000,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.993,3.13
3,3,1004,1978-05-31,1979-02-28,1979-04-30,9.320,1.066,5.657,0.157,9.357000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.878,3.45
4,4,1004,1978-05-31,1979-02-28,1979-05-31,8.811,1.066,5.657,0.149,8.846000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.830,4.24


In [ ]:
df_fill.iloc[:, 34:] = df_fill.iloc[:, 34:].fillna(0) # I do not fill zero for shares outstanding and adjusted shares outstanding column I just drop them in the next line

In [ ]:
df_fill = df_fill.dropna()

In [ ]:
df_fill['log_mcap'] = np.log(df_fill['adj_prccm']*df_fill['adj_cshom']) #creating log market cap column

In [ ]:
df_fill = pd.concat([df_fill, pd.get_dummies(df_fill.iloc[:,8:12], drop_first=True)], axis = 1)
df_fill.head()

,public_date,gvkey,iid,tic,ajexm,prccm,trt1m,cshom,ggroup,gind,gsector,gsubind,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,adate,qdate,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
0,1979-01-31,1004,01,AIR,4.2188,12.000,0.156627,27690000.0,10,10,20,10,2.844411,27690000.0,-0.020833,0.118631,1978-05-31,1978-11-30,10.409,1.236,5.802,0.179,41.009998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.983,3.17,18.181938,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
642,1979-02-28,1004,01,AIR,4.2188,11.750,-0.012917,27690000.0,10,10,20,10,2.785152,27690000.0,0.031915,0.118631,1978-05-31,1978-11-30,10.192,1.236,5.802,0.176,40.155998,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.963,3.23,18.160885,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1286,1979-03-31,1004,01,AIR,4.2188,12.125,0.031915,27690000.0,10,10,20,10,2.874040,27690000.0,-0.092783,0.118631,1978-05-31,1978-11-30,10.517,1.236,5.802,0.181,41.437000,0.032,0.138,0.143,0.164,0.398,0.513,0.487,0.778,3.756,0.669,2.023,0.823,3.042,0.0,0.993,3.13,18.192301,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1933,1979-04-30,1004,01,AIR,4.2188,11.000,-0.092783,27690000.0,10,10,20,10,2.607377,27690000.0,-0.056818,0.118631,1978-05-31,1979-02-28,9.320,1.066,5.657,0.157,9.357000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.878,3.45,18.094927,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2581,1979-05-31,1004,01,AIR,4.2188,10.375,-0.046818,27690000.0,10,10,20,10,2.459230,27690000.0,-0.060241,0.118631,1978-05-31,1979-02-28,8.811,1.066,5.657,0.149,8.846000,0.032,0.143,0.149,0.172,0.422,0.538,0.462,0.779,3.443,0.662,1.960,0.771,3.241,0.0,0.830,4.24,18.036430,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_fill.index = df_fill['public_date']

In [ ]:
df_fill = df_fill['1980':]
df_fill = df_fill.drop(['ggroup', 'gind', 'gsector', 'gsubind', 'adate', 'qdate'], axis = 1)
df_fill.to_pickle('final_data.gz', compression = 'gzip') #creating pickle

##OUTLIER REMOVAL

In [ ]:
df = pd.read_pickle('/content/drive/My Drive/Project_Bank/Data/final_data.gz', compression = 'gzip')

In [ ]:
df.head()

,public_date,gvkey,iid,tic,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
public_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1980-01-31,1980-01-31,1004,01,AIR,3.375,14.375,0.226695,27690000.0,4.259259,27690000.0,0.026087,0.126424,14.258,1.179,5.434,0.243,-16.448999,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.312,2.78,18.585678,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-02-29,1980-02-29,1004,01,AIR,3.375,14.750,0.026087,27690000.0,4.370370,27690000.0,-0.296610,0.125305,14.623,1.179,5.434,0.249,-16.870001,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.345,2.71,18.611429,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-03-31,1980-03-31,1004,01,AIR,3.375,10.375,-0.296610,27690000.0,3.074074,27690000.0,0.000000,0.160163,10.286,1.179,5.434,0.175,-11.866000,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,0.946,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-04-30,1980-04-30,1004,01,AIR,3.375,10.375,0.009639,27690000.0,3.074074,27690000.0,-0.012048,0.156256,9.899,0.734,5.730,0.169,-15.295000,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.862,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-05-31,1980-05-31,1004,01,AIR,3.375,10.250,-0.012048,27690000.0,3.037037,27690000.0,-0.060976,0.155038,10.889,0.734,5.730,0.186,-16.823999,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.948,3.90,18.247465,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.describe()

,gvkey,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
count,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06,1.082653e+06
mean,4.672931e+04,2.237565e+00,7.042133e+01,1.637524e-01,1.221737e+08,3.382678e+02,1.471078e+08,2.417951e+01,4.552958e-01,4.525919e+12,1.392248e+00,1.321753e+12,9.490296e+00,6.963913e+00,-1.269387e+01,3.822964e-03,-4.558139e-01,1.223178e-01,2.625876e-01,-1.889586e+00,2.824086e+00,3.297811e+00,3.602262e+11,5.850901e-01,2.521828e+00,2.137036e+00,1.312344e+01,2.967526e+00,3.682814e+00,1.759641e+00,-inf,5.336585e-01,3.058588e-01,1.190834e-01,1.377173e-03,4.002206e-02,4.626764e-01,2.488443e-01,1.563105e-01,7.758257e-02,8.107861e-03,3.607527e-02,1.040315e-02,5.586277e-02,5.478025e-02,1.598047e-01,1.261281e-01,5.245817e-02,1.377191e-01,1.759493e-01,1.438559e-01,4.138815e-02,3.520149e-02,1.685212e-02,4.716701e-01,1.056386e-01,2.199486e-01,6.964374e-04,8.242530e-02,5.484398e-02,5.311028e-04,4.345067e-02,9.789840e-03,6.314119e-03,4.691254e-03
std,5.814072e+04,1.065545e+01,2.617068e+03,3.431841e+01,6.189186e+08,6.999350e+03,6.823345e+08,9.462452e+03,2.440968e+01,2.719471e+15,5.781104e+01,7.940236e+14,4.633731e+01,4.303447e+01,3.235231e+02,1.686676e+00,6.235790e+01,1.012167e+02,4.585206e-01,2.136830e+02,2.111779e+02,2.331301e+02,2.164006e+14,6.111906e-01,2.188012e+02,8.057243e+00,1.017334e+03,2.145003e+02,6.518973e+00,3.513724e+00,NaN,4.988660e-01,4.607705e-01,3.238868e-01,3.708473e-02,1.960110e-01,4.986052e-01,4.323436e-01,3.631496e-01,2.675137e-01,8.967793e-02,1.864776e-01,1.014639e-01,2.296566e-01,2.275509e-01,3.664250e-01,3.319939e-01,2.229492e-01,3.446051e-01,3.807772e-01,3.509437e-01,1.991864e-01,1.842888e-01,1.287173e-01,4.991970e-01,3.073747e-01,4.142117e-01,2.638092e-02,2.750117e-01,2.276756e-01,2.303956e-02,2.038694e-01,9.845816e-02,7.921021e-02,6.833191e-02
min,1.004000e+03,1.000000e-04,1.000000e-04,-9.999830e-01,0.000000e+00,9.090909e-05,0.000000e+00,-9.999393e-01,0.000000e+00,-5.348530e+05,0.000000e+00,-1.110000e+06,1.000000e-02,-6.631000e+02,-8.683800e+04,-5.910000e+02,-1.440000e+04,-9.565000e+03,-7.672700e+01,-1.945200e+04,-1.290300e+02,-7.245400e+02,-1.840000e+05,0.000000e+00,-4.029000e+04,-1.100000e-02,-4.451400e+02,0.000000e+00,5.600000e-02,0.000000e+00,-inf,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.938000e+0

In [ ]:
df = df.drop(['public_date'], axis = 1).reset_index()
df.head()

,public_date,gvkey,iid,tic,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
0,1980-01-31,1004,01,AIR,3.375,14.375,0.226695,27690000.0,4.259259,27690000.0,0.026087,0.126424,14.258,1.179,5.434,0.243,-16.448999,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.312,2.78,18.585678,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1980-02-29,1004,01,AIR,3.375,14.750,0.026087,27690000.0,4.370370,27690000.0,-0.296610,0.125305,14.623,1.179,5.434,0.249,-16.870001,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.345,2.71,18.611429,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1980-03-31,1004,01,AIR,3.375,10.375,-0.296610,27690000.0,3.074074,27690000.0,0.000000,0.160163,10.286,1.179,5.434,0.175,-11.866000,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,0.946,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1980-04-30,1004,01,AIR,3.375,10.375,0.009639,27690000.0,3.074074,27690000.0,-0.012048,0.156256,9.899,0.734,5.730,0.169,-15.295000,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.862,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1980-05-31,1004,01,AIR,3.375,10.250,-0.012048,27690000.0,3.037037,27690000.0,-0.060976,0.155038,10.889,0.734,5.730,0.186,-16.823999,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.948,3.90,18.247465,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
mcap_inf = list(df['log_mcap'].sort_values()[:408].index)

In [ ]:
df = df.drop(df.index[mcap_inf])
df.describe()

,gvkey,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
count,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06,1.082245e+06
mean,4.668542e+04,2.238041e+00,7.044516e+01,1.638184e-01,1.222197e+08,3.383604e+02,1.471633e+08,2.418398e+01,4.554240e-01,4.527625e+12,1.392228e+00,1.322251e+12,9.487100e+00,6.962829e+00,-1.269823e+01,3.813324e-03,-4.265359e-01,1.223531e-01,2.626435e-01,-1.890535e+00,2.825055e+00,3.298935e+00,3.603620e+11,5.851696e-01,2.522310e+00,2.131036e+00,1.312788e+01,2.968619e+00,3.683210e+00,1.759630e+00,1.928031e+01,5.335548e-01,3.059039e-01,1.191264e-01,1.377692e-03,4.003715e-02,4.626984e-01,2.488346e-01,1.563066e-01,7.760165e-02,8.109994e-03,3.608702e-02,1.036179e-02,5.577850e-02,5.480090e-02,1.597660e-01,1.261711e-01,5.245023e-02,1.377710e-01,1.759398e-01,1.438482e-01,4.140190e-02,3.521384e-02,1.685847e-02,4.717093e-01,1.056267e-01,2.198744e-01,6.966999e-04,8.242866e-02,5.486281e-02,5.313030e-04,4.346705e-02,9.793531e-03,6.316499e-03,4.693022e-03
std,5.810117e+04,1.065738e+01,2.617561e+03,3.432487e+01,6.190363e+08,7.000657e+03,6.824516e+08,9.464234e+03,2.441428e+01,2.719983e+15,5.782179e+01,7.941732e+14,4.634001e+01,4.301013e+01,3.235840e+02,1.686992e+00,6.043179e+01,1.012357e+02,4.585901e-01,2.137232e+02,2.112177e+02,2.331740e+02,2.164413e+14,6.112744e-01,2.188424e+02,7.921670e+00,1.017526e+03,2.145407e+02,6.518579e+00,3.513800e+00,2.758120e+00,4.988730e-01,4.607894e-01,3.239374e-01,3.709171e-02,1.960464e-01,4.986069e-01,4.323380e-01,3.631459e-01,2.675438e-01,8.968963e-02,1.865068e-01,1.012642e-01,2.294936e-01,2.275913e-01,3.663891e-01,3.320422e-01,2.229333e-01,3.446597e-01,3.807691e-01,3.509359e-01,1.992180e-01,1.843200e-01,1.287411e-01,4.991992e-01,3.073594e-01,4.141616e-01,2.638589e-02,2.750168e-01,2.277124e-01,2.304390e-02,2.039061e-01,9.847653e-02,7.922504e-02,6.834473e-02
min,1.004000e+03,1.000000e-04,1.000000e-04,-9.999830e-01,0.000000e+00,9.090909e-05,7.000000e-01,-9.999393e-01,0.000000e+00,-5.348530e+05,0.000000e+00,-1.110000e+06,1.000000e-02,-6.631000e+02,-8.683800e+04,-5.910000e+02,-1.440000e+04,-9.565000e+03,-7.672700e+01,-1.945200e+04,-1.290300e+02,-7.245400e+02,-1.840000e+05,0.000000e+00,-4.029000e+04,-1.100000e-02,-4.451400e+02,0.000000e+00,5.600000e-02,0.000000e+00,1.386294e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0

In [ ]:
df = df[df['trt1m'] < 5]

,public_date,gvkey,iid,tic,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
0,1980-01-31,1004,01,AIR,3.375,14.3750,0.226695,27690000.0,4.259259,27690000.0,0.026087,0.126424,14.258000,1.179,5.434,0.243,-16.448999,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.312,2.78,18.585678,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1980-02-29,1004,01,AIR,3.375,14.7500,0.026087,27690000.0,4.370370,27690000.0,-0.296610,0.125305,14.623000,1.179,5.434,0.249,-16.870001,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.345,2.71,18.611429,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1980-03-31,1004,01,AIR,3.375,10.3750,-0.296610,27690000.0,3.074074,27690000.0,0.000000,0.160163,10.286000,1.179,5.434,0.175,-11.866000,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,0.946,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1980-04-30,1004,01,AIR,3.375,10.3750,0.009639,27690000.0,3.074074,27690000.0,-0.012048,0.156256,9.899000,0.734,5.730,0.169,-15.295000,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.862,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1980-05-31,1004,01,AIR,3.375,10.2500,-0.012048,27690000.0,3.037037,27690000.0,-0.060976,0.155038,10.889000,0.734,5.730,0.186,-16.823999,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.948,3.90,18.247465,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082648,2019-09-30,34618,01,BCOW,1.000,9.6400,0.022974,4877000.0,9.640000,4877000.0,0.012614,0.035183,-44.437000,1.287,11.531,2.336,-16.106001,0.025,0.009,-0.004,0.049,0.031,0.783,0.428,0.428,6.164,0.902,9.225,0.000,0.332,0.0,0.817,0.00,17.665962,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1082649,2019-10-31,34618,01,BCOW,1.000,9.7616,0.012614,4877000.0,9.761600,4877000.0,0.050033,0.035183,-44.997002,1.287,11.531,2.365,-16.309000,0.025,0.009,-0.004,0.049,0.031,0.783,0.428,0.428,6.164,0.902,9.225,0.000,0.332,0.0,0.828,0.00,17.678497,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1082650,2019-11-30,34618,01,BCOW,1.000,10.2500,0.050033,4877000.0,10.250000,4877000.0,0.051707,0.035183,-54.580002,1.289,12.258,2.422,-12.444000,0.034,0.009,0.000,0.049,0.032,0.801,0.285,0.285,4.357,0.889,8.023,0.000,0.310,0.0,0.845,0.00,17.727318,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1082651,2019-12-31,34618,01,BCOW,1.000,10.7800,0.051707,4877000.0,10.780000,4877000.0,0.104824,0.035183,-58.111000,1.289,12.258,2.578,-13.250000,0.034,0.009,0.000,0.049,0.032,0.801,0.285,0.285,4.357,0.889,8.023,0.000,0.310,0.0,0.899,0.00,17.777733,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = df[df['fut_1m_r'] < 5]
df.describe()

,gvkey,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
count,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06,1.080127e+06
mean,4.661878e+04,2.240455e+00,7.057343e+01,1.509796e-02,1.218186e+08,3.379720e+02,1.468323e+08,1.393682e-02,3.975097e-01,4.536504e+12,1.387185e+00,1.324844e+12,9.473761e+00,6.975207e+00,-1.266019e+01,4.732908e-03,-4.244770e-01,1.241504e-01,2.628794e-01,-1.895008e+00,2.829605e+00,3.302578e+00,3.610687e+11,5.848489e-01,2.525575e+00,2.131546e+00,1.314797e+01,2.963246e+00,3.681873e+00,1.761031e+00,1.929489e+01,5.335715e-01,3.057761e-01,1.192138e-01,1.375764e-03,4.006288e-02,4.628271e-01,2.486022e-01,1.563973e-01,7.759273e-02,8.105528e-03,3.611612e-02,1.035897e-02,5.573604e-02,5.478337e-02,1.597969e-01,1.261990e-01,5.244661e-02,1.376347e-01,1.761108e-01,1.438377e-01,4.134977e-02,3.524030e-02,1.686468e-02,4.716131e-01,1.057024e-01,2.198880e-01,6.980661e-04,8.244123e-02,5.488706e-02,5.314190e-04,4.343563e-02,9.799774e-03,6.319627e-03,4.683708e-03
std,5.806668e+04,1.066719e+01,2.620121e+03,2.128593e-01,6.122264e+08,6.954968e+03,6.762474e+08,2.141235e-01,2.310405e+01,2.722649e+15,5.766407e+01,7.949513e+14,4.632457e+01,4.302454e+01,3.235861e+02,1.685462e+00,6.048932e+01,1.013348e+02,4.581763e-01,2.139327e+02,2.114245e+02,2.334020e+02,2.166534e+14,6.049074e-01,2.190249e+02,7.927444e+00,1.018522e+03,2.147382e+02,6.513862e+00,3.512253e+00,2.736679e+00,4.988719e-01,4.607356e-01,3.240400e-01,3.706579e-02,1.961068e-01,4.986165e-01,4.322029e-01,3.632318e-01,2.675297e-01,8.966513e-02,1.865792e-01,1.012505e-01,2.294114e-01,2.275570e-01,3.664178e-01,3.320737e-01,2.229260e-01,3.445164e-01,3.809146e-01,3.509253e-01,1.990980e-01,1.843867e-01,1.287644e-01,4.991938e-01,3.074565e-01,4.141708e-01,2.641173e-02,2.750359e-01,2.277598e-01,2.304641e-02,2.038358e-01,9.850760e-02,7.924453e-02,6.827720e-02
min,1.004000e+03,1.000000e-04,1.000000e-04,-9.999830e-01,2.140000e+02,9.090909e-05,7.000000e-01,-9.999393e-01,0.000000e+00,-5.348530e+05,0.000000e+00,-1.110000e+06,1.000000e-02,-6.631000e+02,-8.683800e+04,-5.910000e+02,-1.440000e+04,-9.565000e+03,-7.672700e+01,-1.945200e+04,-1.290300e+02,-7.245400e+02,-1.840000e+05,0.000000e+00,-4.029000e+04,-1.100000e-02,-4.451400e+02,0.000000e+00,5.600000e-02,0.000000e+00,1.589235e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0

In [ ]:
df.to_pickle('final_data.gz', compression = 'gzip')

# Creating Sequences

In [ ]:
df.head()

,public_date,gvkey,iid,tic,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
0,1980-01-31,1004,01,AIR,3.375,14.375,0.226695,27690000.0,4.259259,27690000.0,0.026087,0.126424,14.258,1.179,5.434,0.243,-16.448999,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.312,2.78,18.585678,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1980-02-29,1004,01,AIR,3.375,14.750,0.026087,27690000.0,4.370370,27690000.0,-0.296610,0.125305,14.623,1.179,5.434,0.249,-16.870001,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.345,2.71,18.611429,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1980-03-31,1004,01,AIR,3.375,10.375,-0.296610,27690000.0,3.074074,27690000.0,0.000000,0.160163,10.286,1.179,5.434,0.175,-11.866000,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,0.946,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1980-04-30,1004,01,AIR,3.375,10.375,0.009639,27690000.0,3.074074,27690000.0,-0.012048,0.156256,9.899,0.734,5.730,0.169,-15.295000,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.862,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1980-05-31,1004,01,AIR,3.375,10.250,-0.012048,27690000.0,3.037037,27690000.0,-0.060976,0.155038,10.889,0.734,5.730,0.186,-16.823999,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.948,3.90,18.247465,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.index = df['public_date']
df.head()

,public_date,gvkey,iid,tic,ajexm,prccm,trt1m,cshom,adj_prccm,adj_cshom,fut_1m_r,1yr_ret_vol,CAPEI,bm,evm,ps,pcf,cfm,roa,roe,roce,GProf,equity_invcap,debt_invcap,totdebt_invcap,debt_ebitda,debt_assets,de_ratio,quick_ratio,sale_invcap,rd_sale,ptb,divyield,log_mcap,ggroup_10,ggroup_20,ggroup_30,ggroup_40,ggroup_50,gind_10,gind_20,gind_30,gind_40,gind_50,gind_60,gind_70,gsector_10,gsector_15,gsector_20,gsector_25,gsector_30,gsector_35,gsector_40,gsector_45,gsector_50,gsector_55,gsector_60,gsubind_10,gsubind_15,gsubind_20,gsubind_25,gsubind_30,gsubind_40,gsubind_45,gsubind_50,gsubind_60,gsubind_70,gsubind_80
public_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1980-01-31,1980-01-31,1004,01,AIR,3.375,14.375,0.226695,27690000.0,4.259259,27690000.0,0.026087,0.126424,14.258,1.179,5.434,0.243,-16.448999,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.312,2.78,18.585678,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-02-29,1980-02-29,1004,01,AIR,3.375,14.750,0.026087,27690000.0,4.370370,27690000.0,-0.296610,0.125305,14.623,1.179,5.434,0.249,-16.870001,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,1.345,2.71,18.611429,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-03-31,1980-03-31,1004,01,AIR,3.375,10.375,-0.296610,27690000.0,3.074074,27690000.0,0.000000,0.160163,10.286,1.179,5.434,0.175,-11.866000,0.038,0.152,0.157,0.184,0.426,0.538,0.462,0.793,3.374,0.665,1.983,0.782,3.149,0.0,0.946,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-04-30,1980-04-30,1004,01,AIR,3.375,10.375,0.009639,27690000.0,3.074074,27690000.0,-0.012048,0.156256,9.899,0.734,5.730,0.169,-15.295000,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.862,3.86,18.259586,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1980-05-31,1980-05-31,1004,01,AIR,3.375,10.250,-0.012048,27690000.0,3.037037,27690000.0,-0.060976,0.155038,10.889,0.734,5.730,0.186,-16.823999,0.038,0.155,0.167,0.187,0.420,0.534,0.466,0.795,3.400,0.664,1.980,0.784,3.080,0.0,0.948,3.90,18.247465,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['1986-01'].ggroup_40.to_csv('jkl.csv')